In [145]:
import numpy as np
from sklearn import decomposition
from scipy import linalg
import matplotlib.pyplot as plt
import pandas as pd
import nltk
import gensim

In [146]:
%matplotlib inline
np.set_printoptions(suppress=True)

In [147]:
data = pd.read_csv('data.csv')
data

,Id,Channel,Subscribers,Title,CC,URL,Released,Views,Category,Transcript,Length
0,FozCkl1xj-w,JRE Clips,6.28M subscribers,Former CIA Agent Breaks Down Jeffrey Epstein Case,0,https://www.youtube.com/watch?v=FozCkl1xj-w,2 years ago,7.9M views,Blog,the Joe Rogan experience well how about the ot...,13:32
1,RN8yoi-e2yc,Mythical Kitchen,1.9M subscribers,$420 Pizza Hut Stuffed Crust Pizza | Fancy Fas...,1,https://www.youtube.com/watch?v=RN8yoi-e2yc,NaN,2.7M views,Food,"- Oh, that's dirty.\r\n- Wow! - Whoa.\r\n- You...",24:26
2,IugcIAAZJ2M,Munchies,4.59M subscribers,The Iconic $1 Pizza Slice of NYC | Street Food...,0,https://www.youtube.com/watch?v=IugcIAAZJ2M,2 years ago,11M views,Food,if you want good pizza come to st marks it's t...,7:51
3,JiEO6F8i0eU,Parks and Recreation,282K subscribers,Ron Swanson: The Papa of Pawnee | Parks and Re...,0,https://www.youtube.com/watch?v=JiEO6F8i0eU,3 years ago,2.3M views,"Entertainment,Comedy",April where have you been over two phone calls...,10:06
4,1T4XMNN4bNM,Vsauce,17.4M subscribers,What's The Most Dangerous Place on Earth?,1,https://www.youtube.com/watch?v=1T4XMNN4bNM,9 years ago,21M views,Science,"Hey, Vsauce. Michael here. 93% of all the\r\nh...",9:29
...,...,...,...,...,...,...,...,...,...,...,...
2510,DonD68byO0o,Parks and Recreation,282K subscribers,Best of Andy Dwyer | Parks and Recreation,0,https://www.youtube.com/watch?v=DonD68byO0o,3 years ago,2.8M views,"Entertainment,Comedy",oh I guess while you get your medicine I'll ju...,10:05
2511,iaEcZW0eyEg,Hell's Kitchen,1.5M subscribers,Gordon Ramsay's Most Savage Moments | Hell's K...,0,https://www.youtube.com/watch?v=iaEcZW0eyEg,11 months ago,6.4M views,Food,southern girl keisha hopes to get the women on...,8:33
2512,sW9npZVpiMI,Joma Tech,1.59M subscribers,why you NEED math for programming,0,https://www.youtube.com/watch?v=sW9npZVpiMI,NaN,4.5M views,"Tech,Comedy",[Music] okay [Music] jesus nope not jesus it's...,5:03
2513,yl3MiR7XXAM,jacksepticeye,28M subscribers,Happy Wheels - Part 1 | THIS GAME IS MY BITCH,1,https://www.youtube.com/watch?v=yl3MiR7XXAM,8 years ago,20M views,VideoGames,"Top o' the mornin' to ya, laddies! My name i...",13:32


In [148]:
data = data.Title.str.split(" ")
data

0       [Former, CIA, Agent, Breaks, Down, Jeffrey, Ep...
1       [$420, Pizza, Hut, Stuffed, Crust, Pizza, |, F...
2       [The, Iconic, $1, Pizza, Slice, of, NYC, |, St...
3       [Ron, Swanson:, The, Papa, of, Pawnee, |, Park...
4       [What's, The, Most, Dangerous, Place, on, Earth?]
                              ...                        
2510    [Best, of, Andy, Dwyer, |, Parks, and, Recreat...
2511    [Gordon, Ramsay's, Most, Savage, Moments, |, H...
2512             [why, you, NEED, math, for, programming]
2513    [Happy, Wheels, -, Part, 1, |, THIS, GAME, IS,...
2514    [The, Smallest, Car, in, the, World, at, the, ...
Name: Title, Length: 2515, dtype: object

In [149]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Erickson\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Erickson\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [150]:
lemma = WordNetLemmatizer()

In [151]:
data = data.apply(lambda doc: [lemma.lemmatize(word) for word in doc])
data

0       [Former, CIA, Agent, Breaks, Down, Jeffrey, Ep...
1       [$420, Pizza, Hut, Stuffed, Crust, Pizza, |, F...
2       [The, Iconic, $1, Pizza, Slice, of, NYC, |, St...
3       [Ron, Swanson:, The, Papa, of, Pawnee, |, Park...
4       [What's, The, Most, Dangerous, Place, on, Earth?]
                              ...                        
2510    [Best, of, Andy, Dwyer, |, Parks, and, Recreat...
2511    [Gordon, Ramsay's, Most, Savage, Moments, |, H...
2512             [why, you, NEED, math, for, programming]
2513    [Happy, Wheels, -, Part, 1, |, THIS, GAME, IS,...
2514    [The, Smallest, Car, in, the, World, at, the, ...
Name: Title, Length: 2515, dtype: object

In [152]:
from nltk.corpus import stopwords

In [153]:
swords = stopwords.words('english')

In [154]:
data = data.apply(lambda doc: [word for word in doc if word not in swords])
data

0       [Former, CIA, Agent, Breaks, Down, Jeffrey, Ep...
1       [$420, Pizza, Hut, Stuffed, Crust, Pizza, |, F...
2       [The, Iconic, $1, Pizza, Slice, NYC, |, Street...
3       [Ron, Swanson:, The, Papa, Pawnee, |, Parks, R...
4           [What's, The, Most, Dangerous, Place, Earth?]
                              ...                        
2510            [Best, Andy, Dwyer, |, Parks, Recreation]
2511    [Gordon, Ramsay's, Most, Savage, Moments, |, H...
2512                            [NEED, math, programming]
2513    [Happy, Wheels, -, Part, 1, |, THIS, GAME, IS,...
2514    [The, Smallest, Car, World, BBC, -, Top, Gear,...
Name: Title, Length: 2515, dtype: object

In [155]:
from gensim.corpora.dictionary import Dictionary

In [156]:
dct = Dictionary(data)

In [157]:
dct.filter_extremes(no_below=5, no_above=0.5)

In [158]:
len(dct.token2id)

635

In [159]:
bow = data.apply(dct.doc2bow)
bow

0                                        [(0, 1), (1, 1)]
1       [(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2...
2               [(4, 1), (7, 1), (8, 1), (9, 1), (10, 1)]
3            [(8, 1), (10, 1), (11, 1), (12, 1), (13, 1)]
4                             [(10, 1), (14, 1), (15, 1)]
                              ...                        
2510                 [(8, 1), (11, 1), (12, 1), (173, 1)]
2511    [(5, 1), (8, 2), (14, 1), (22, 1), (61, 1), (1...
2512                                                   []
2513    [(8, 1), (25, 1), (61, 1), (180, 1), (249, 1),...
2514    [(10, 1), (25, 2), (98, 2), (103, 1), (119, 1)...
Name: Title, Length: 2515, dtype: object

In [160]:
from gensim.models import LdaModel

In [161]:
lda = LdaModel(bow, num_topics=20, id2word=dct)

In [162]:
for i in range(20):
    print("Topic_"+str(i), lda.print_topic(i))
    print("")

Topic_0 0.053*"|" + 0.050*"Spicy" + 0.045*"Hot" + 0.044*"While" + 0.042*"Wings" + 0.042*"Eating" + 0.041*"Ones" + 0.034*"The" + 0.030*"Voice" + 0.024*"6"

Topic_1 0.092*"|" + 0.065*"-" + 0.031*"Rogan" + 0.031*"Joe" + 0.025*"1" + 0.024*"#1" + 0.020*"Fridman" + 0.020*"Lex" + 0.019*"Gaming" + 0.018*"Part"

Topic_2 0.157*"-" + 0.080*"The" + 0.043*"&" + 0.036*"Peele" + 0.036*"Key" + 0.034*"|" + 0.022*"US" + 0.020*"BBC" + 0.017*"It" + 0.016*"Office"

Topic_3 0.119*"|" + 0.036*"&" + 0.033*"Hell's" + 0.032*"Kitchen" + 0.022*"James" + 0.022*"Is" + 0.022*"Fast" + 0.020*"Building" + 0.020*"Ones" + 0.019*"While"

Topic_4 0.052*"10" + 0.039*"This" + 0.035*"In" + 0.035*"-" + 0.034*"Minutes" + 0.033*"That" + 0.027*"Is" + 0.026*"How" + 0.024*"Black" + 0.023*"The"

Topic_5 0.100*"-" + 0.076*"The" + 0.044*"OverSimplified" + 0.038*"(Part" + 0.034*"Honest" + 0.034*"Trailers" + 0.032*"Making" + 0.031*"2)" + 0.020*"SHROUD" + 0.019*"I"

Topic_6 0.108*"|" + 0.067*"The" + 0.059*"What" + 0.047*"Doctor" + 0.034*